In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc = nn.Linear(n_features, 1)

    def forward(self, x):
        return self.fc(x)

df = pd.read_csv('./dataset/room_info_reform.csv')
df = df[df['target']!='uninspected']

# データフレームを訓練データとテストデータに分割
X = df.drop(['room_num', 'target'], axis=1)
y = df['target'].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# いくつかのモデルを試す
models = [
    {"name": "RF", "model": RandomForestRegressor()}
]

for m in models:
    model = m["model"]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Model: {m['name']}")
    print(f"Mean Squared Error: {mse}")

    print("\n-----------------------------------\n")

# PyTorch part starts here

X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model = Net(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

num_epochs = 10
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

model.eval()
with torch.no_grad():
    outputs = model(X_test)
    mse = criterion(outputs, y_test).item()
    print(f"Model: PyTorch NN")
    print(f"Mean Squared Error: {mse}")
    print("\n-----------------------------------\n")
    

Model: RF
Mean Squared Error: 151.25624999999997

-----------------------------------

Model: PyTorch NN
Mean Squared Error: 563.00341796875

-----------------------------------



In [2]:
torch.save(model.state_dict(), './learned_model/torch.pth')
